In [ ]:
import numpy as np
import skimage
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import distance
import scipy 
from starter import *
from skimage import transform

def plot_inlier_matches(ax, img1, img2, inliers):
    """
    Plot the matches between two images according to the matched keypoints
    :param ax: plot handle
    :param img1: left image
    :param img2: right image
    :inliers: x,y in the first image and x,y in the second image (Nx4)
    """
    inliers = inliers.reshape(-1,4)
    print(inliers.shape)
    res = np.hstack([img1, img2])
    ax.set_aspect('equal')
    ax.imshow(res, cmap='gray')
    
    ax.plot(inliers[:,0], inliers[:,1], '+r')
    ax.plot(inliers[:,2] + img1.shape[1], inliers[:,3], '+r')
    ax.plot([inliers[:,0], inliers[:,2] + img1.shape[1]],
            [inliers[:,1], inliers[:,3]], 'r', linewidth=0.4)
    ax.axis('off')
    
#Load both Images, convert to double and grayscale
left_image = cv2.imread("pier1.jpg")
middle_image = cv2.imread("pier2.jpg")
right_image = cv2.imread("pier3.jpg")

left_gray = cv2.cvtColor(left_image, cv2.COLOR_BGR2GRAY)
middle_gray = cv2.cvtColor(middle_image, cv2.COLOR_BGR2GRAY)
right_gray = cv2.cvtColor(right_image, cv2.COLOR_BGR2GRAY)


def RANSAC_loop(matches, num_iterations, inlier_threshold):
    """
    RANSAC loop:
    1. Randomly select a seed group of matches (4 matches to start)
    2. Compute transformation from seed group
    3. Find inliers to this transformation
    4. If the number of inliers is sufficiently large, re-compute least-squares
    estimate of transformation on all of the inliers
    • At the end, keep the transformation with the largest number of inliers
    """
    max_inliers = 0 
    best_H = None
    all_inliers = None
    for i in range(num_iterations):
        #1
        seed_indices = np.random.choice(matches.shape[0], 4, replace=False)
        seed_group = matches[seed_indices]

        points_left = np.array([match[0] for match in seed_group])
        points_right = np.array([match[1] for match in seed_group])

        #2 Compute Homography
        A = []
        for i in range(len(points_left)):
            x1, y1 = points_left[i]
            x2, y2 = points_right[i]

            A.append([0, 0, 0, x1, y1, 1, -y2 * x1, -y2 * y1, -y2]) 
            A.append([x1, y1, 1, 0, 0, 0, -x2 * x1, -x2 * y1, -x2])  
        
        A = np.array(A)
        U, S, V = np.linalg.svd(A)
        H = V[-1] 
        H = H.reshape(3, 3) 

        points1_homogeneous = cv2.convertPointsToHomogeneous(np.array([m[0] for m in matches])).reshape(-1, 3) #Convert to homogeneous coordinates
        transformed_points = (H @ points1_homogeneous.T).T
        transformed_points /= transformed_points[:, 2].reshape(-1, 1)  # Convert from homogeneous coordinates

        # Calculate the distances to the corresponding points in right image
        points2_homogeneous = np.array([m[1] for m in matches])
        distances = np.sqrt(np.sum((transformed_points[:, :2] - points2_homogeneous) ** 2, axis=1))

        # Determine inliers
        inlier_mask = distances < inlier_threshold
        inliers_count = np.sum(inlier_mask)

        #4
        if inliers_count > max_inliers:
            max_inliers = inliers_count
            best_H = H
            all_inliers = matches[inlier_mask]


    # Calculate average residual for inliers
    if all_inliers is not None:
        inlier_points1 = np.array([m[0] for m in all_inliers])
        inlier_points2 = np.array([m[1] for m in all_inliers])
        
        transformed_inlier_points1 = (best_H @ cv2.convertPointsToHomogeneous(inlier_points1).reshape(-1,3).T).T
        transformed_inlier_points1 /= transformed_inlier_points1[:, 2].reshape(-1, 1)  # Normalize
        
        residuals = np.sqrt(np.sum((transformed_inlier_points1[:, :2] - inlier_points2) ** 2, axis=1))
        average_residual = np.mean(residuals)
    else:
        average_residual = None

    return best_H, max_inliers, average_residual, all_inliers



In [ ]:
# 3&4: Extract keypoints and neighborhoods using SIFT descriptors
def get_keypoints_and_matches(left_gray, right_gray):
    sift_left = cv2.SIFT_create()
    sift_middle = cv2.SIFT_create()
    kp_left, des_left= sift_left.detectAndCompute(left_gray, None)
    kp_middle, des_right = sift_middle.detectAndCompute(right_gray, None)

    kp_left_coords = np.array([kp.pt for kp in kp_left])
    kp_right_coords = np.array([kp.pt for kp in kp_middle])

    #5 Compute Distances between every desciptor of one image to those of the other. 
    eucdistance = distance.cdist(des_left, des_right, "sqeuclidean")


    #Select Putative matches by selecting top few hundred descriptor pairs with smallest pairwise distances
    k = 150
    flat_indices = np.argsort(eucdistance, axis=None)[:k]
    row_indices, col_indices = np.unravel_index(flat_indices, eucdistance.shape)
    filtered_matches_top_k = np.array([kp_left[row_indices[i]], kp_right[col_indices[i]]] for i in range(k))
    filtered_matches = np.array([(kp_left_coords[row_indices[i]], kp_right_coords[col_indices[i]]) for i in range(k)])
    return filtered_matches

filtered_matches = get_keypoints_and_matches(left_gray, middle_gray)

print("Top-k matches found:", len(filtered_matches))

while True:
    inlier_threshold = 60
    num_iterations = 1000
    best_H,  max_inliers, average_residual, all_inliers = RANSAC_loop(filtered_matches, num_iterations=num_iterations, inlier_threshold=inlier_threshold)
    if average_residual < 0.4:
        break

fig, ax = plt.subplots(figsize=(20,10))
plot_inlier_matches(ax, left_image, middle_image, all_inliers)
plt.show()
print(f"Number of inliers: {max_inliers}")

def stitch_image(left_image, right_image, best_H):
    h, w = left_image.shape[:2]
    h2, w2 = right_image.shape[:2]
    corners = np.array([[0, 0], [w, 0], [0, h], [w, h]])

    transform_object = skimage.transform.ProjectiveTransform(best_H)
    warped_corners = transform_object(corners)

    min_coords = warped_corners.min(axis=0)
    max_coords = warped_corners.max(axis=0)

    translation = transform.AffineTransform(translation=-min_coords)
    transformed_H = translation.params @ best_H

    output_shape = (
        int(max(max_coords[1], h2) - min(min_coords[1], 0)),
        int(max(max_coords[0], w2) - min(min_coords[0], 0))
    )


    canvas = np.zeros((output_shape[0], output_shape[1], 3), dtype=np.uint8)
    left_image = cv2.cvtColor(left_image, cv2.COLOR_BGR2RGB)
    right_image = cv2.cvtColor(right_image, cv2.COLOR_BGR2RGB)
    for channel in range(3):
        warped_left_image = transform.warp(left_image[:, :, channel].astype(np.float32)/255, transform.ProjectiveTransform(transformed_H).inverse, output_shape=output_shape)
        warped_left_image = (warped_left_image * 255).astype(np.uint8)
        canvas[:, :, channel] = warped_left_image
        x_offset = int(np.abs(min_coords[0]))
        y_offset = int(np.abs(min_coords[1]))
        # warped_mask = (warped_left_image > 0)

        for y in range(h2):
            for x in range(w2):
                canvas_y = y + y_offset
                canvas_x = x + x_offset
                if canvas[canvas_y, canvas_x, channel] > 0:
                    # canvas[canvas_y, canvas_x] = (canvas[canvas_y, canvas_x] + right_gray[y, x]) / 2
                    canvas[canvas_y, canvas_x, channel] = canvas[canvas_y, canvas_x, channel]
                else:
                    canvas[canvas_y, canvas_x, channel] = right_image[y, x, channel]
    # canvas[int(np.abs(min_coords[1])):int(np.abs(min_coords[1]))+h2, int(np.abs(min_coords[0])):] = right_gray
    canvas_final = cv2.cvtColor(canvas, cv2.COLOR_RGB2BGR)
    return canvas_final

canvas_final = stitch_image(left_image, middle_image, best_H)

cv2.imwrite("Pier1and2.jpg", canvas_final)

In [16]:
combined_image = cv2.imread("Pier1and2.jpg")
combined_gray = cv2.cvtColor(combined_image, cv2.COLOR_BGR2GRAY)

filtered_matches = get_keypoints_and_matches(combined_gray, right_gray)

while True:
    inlier_threshold = 60
    num_iterations = 1000
    best_H,  max_inliers, average_residual, all_inliers = RANSAC_loop(filtered_matches, num_iterations=num_iterations, inlier_threshold=inlier_threshold)
    print(f"Average Residual: {average_residual}")
    if average_residual < 0.4:
        break
print(f"Number of Inliers for complete stitching: {max_inliers}")



canvas_final_final = stitch_image(combined_image, right_image, best_H)
cv2.imwrite("P2_Canvas.jpg", canvas_final_final)

Average Residual: 0.3125340286524681
Number of Inliers for complete stitching: 150


True